In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, set_seed
from datasets import *
import numpy as np
import pandas as pd
import torch
import ast

In [2]:
SEED = 42
set_seed(SEED)

local_models_path = '../models/T5'

results = pd.DataFrame(columns=['Model', 'Train Data', 'Data Type', 'Bert.Precision', 'Bert.Recall', 'Bert.F1', 'BLEU'])

In [3]:
def get_answer(question, context, tokenizer, model):
    input_text = "question: %s  context: %s" % (question, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], attention_mask=features['attention_mask'])

    return tokenizer.decode(output[0], skip_special_tokens=True)

## Load all data

2020

In [4]:
# Load the dataset from file and split it into train and test datasets
data_2020_full = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-2020-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_full["test"] = data_2020_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data_2020_full["test"] = data_2020_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_full["test"] = data_2020_full["test"].remove_columns(["text", "answer_start"])
# get ground truth answers
test_data_2020_full = data_2020_full["test"]
gt_answers_2020_full = [temp["answers"]["text"][0] for temp in test_data_2020_full]

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-dc9275c3697e5cc0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc9275c3697e5cc0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0b40be795cd765b7.arrow and C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc9275c3697e5cc0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-3e1cf7e5eca93f71.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc9275c3697e5cc0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-1e92815268136ea2.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc9275c3697e5cc0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-21e74bfcddde162b.arrow
Loading cached process

2022

In [5]:
data_2022_full = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-2022-squad-format.csv",
                    delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_full["test"] = data_2022_full["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data_2022_full["test"] = data_2022_full["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_full["test"] = data_2022_full["test"].remove_columns(["text", "answer_start"])

test_data_2022_full = data_2022_full["test"]
gt_answers_2022_full = [temp["answers"]["text"][0] for temp in test_data_2022_full]

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-a8af1b4c8d81fb1c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-a8af1b4c8d81fb1c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-7d8841dd72615495.arrow and C:\Users\rjutr\.cache\huggingface\datasets\csv\default-a8af1b4c8d81fb1c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8b1142ae44904f4e.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-a8af1b4c8d81fb1c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-36096c12f970e2cd.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-a8af1b4c8d81fb1c\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b621ad6a176502d0.arrow
Loading cached process

2020 + 2022

In [6]:
data_2020_2022 = load_dataset('csv', data_files="../../data/clean/sustainability-report-2042-squad-format.csv",
                                delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2020_2022["test"] = data_2020_2022["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data_2020_2022["test"] = data_2020_2022["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2020_2022["test"] = data_2020_2022["test"].remove_columns(["text", "answer_start"])
# get ground truth answers
test_data_2020_2022 = data_2020_2022["test"]
gt_answers_2020_2022 = [temp["answers"]["text"][0] for temp in test_data_2020_2022]

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-dc46deea403e6d7a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc46deea403e6d7a\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-8c4226c3883b9f9f.arrow and C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc46deea403e6d7a\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-24d9770f6403af3d.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc46deea403e6d7a\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-47e77242f5a0d387.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-dc46deea403e6d7a\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-cb788cd2e517f2b5.arrow
Loading cached process

2022 handwritten

In [7]:
data_2022_handwritten = load_dataset('csv', data_files=f"../../data/clean/QA_SR_2022_Expert-squad-format.csv",
                                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)

# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data_2022_handwritten["test"] = data_2022_handwritten["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data_2022_handwritten["test"] = data_2022_handwritten["test"].remove_columns(["text", "answer_start"])

test_data_2022_handwritten = data_2022_handwritten["test"]
gt_answers_2022_handwritten = [temp["answers"]["text"][0] for temp in test_data_2022_handwritten]

Found cached dataset csv (C:/Users/rjutr/.cache/huggingface/datasets/csv/default-094de926302c4454/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached split indices for dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-094de926302c4454\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-a6b47fc8c478d969.arrow and C:\Users\rjutr\.cache\huggingface\datasets\csv\default-094de926302c4454\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4ae4e0308bfc9ae2.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-094de926302c4454\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b9d36a6ac093f305.arrow
Loading cached processed dataset at C:\Users\rjutr\.cache\huggingface\datasets\csv\default-094de926302c4454\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-c31eb97e51e66589.arrow
Loading cached process

## SIMPLE EVALUATION

In [8]:
import evaluate
bertscore = evaluate.load("bertscore")
bleu = evaluate.load("bleu")

### Small

In [9]:
model_name = "mrm8488/t5-small-finetuned-squadv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

answers_2020 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")
# print(f"Bertscore results 2022 handwritten\nF1: {np.array(bert_results_2022_handwritten['f1']).mean()}, Precision: {np.array(bert_results_2022_handwritten['precision']).mean()}, Recall: {np.array(bert_results_2022_handwritten['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2020-2022\n{bleu_results_2020_2022}")
# print(f"Bleu results 2022 handwritten\n{bleu_results_2022_handwritten}")


# add results to dataframe
results.loc[len(results)] = ['t5-small', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-small', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Small - finetuned

In [10]:
model_name_2020 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2020-full"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-full"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

model_name_2022_handwritten = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = AutoTokenizer.from_pretrained(model_name_2022_handwritten, local_files_only=True)
model_2022_handwritten = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022_handwritten, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer_2022_handwritten, model_2022_handwritten) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")
# print(f"Bertscore results 2022 handwritten\nF1: {np.array(bert_results_2022_handwritten['f1']).mean()}, Precision: {np.array(bert_results_2022_handwritten['precision']).mean()}, Recall: {np.array(bert_results_2022_handwritten['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2022 handwritten\n{bleu_results_2022_handwritten}")


# add results to dataframe
results.loc[len(results)] = ['t5-small', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Small - finetuned - train set halved

In [11]:
model_name_2020 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2020-2022\n{bleu_results_2020_2022}")


# add results to dataframe
results.loc[len(results)] = ['t5-small', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-small', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-small', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Base

In [12]:
model_name = "mrm8488/t5-base-finetuned-squadv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

answers_2020 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer, model) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")
# print(f"Bertscore results 2022 handwritten\nF1: {np.array(bert_results_2022_handwritten['f1']).mean()}, Precision: {np.array(bert_results_2022_handwritten['precision']).mean()}, Recall: {np.array(bert_results_2022_handwritten['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2020-2022\n{bleu_results_2020_2022}")
# print(f"Bleu results 2022 handwritten\n{bleu_results_2022_handwritten}")


# add results to dataframe
results.loc[len(results)] = ['t5-base', None, '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', None, '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', None, '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-base', None, '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Base - finetuned

In [13]:
model_name_2020 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2020-full"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-full"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

model_name_2022_handwritten = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-handwritten"
tokenizer_2022_handwritten = AutoTokenizer.from_pretrained(model_name_2022_handwritten, local_files_only=True)
model_2022_handwritten = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022_handwritten, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]
answers_2022_handwritten = [get_answer(question, context, tokenizer_2022_handwritten, model_2022_handwritten) for question, context in zip(test_data_2022_handwritten["question"], test_data_2022_handwritten["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
bert_results_2022_handwritten = bertscore.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")
# print(f"Bertscore results 2022 handwritten\nF1: {np.array(bert_results_2022_handwritten['f1']).mean()}, Precision: {np.array(bert_results_2022_handwritten['precision']).mean()}, Recall: {np.array(bert_results_2022_handwritten['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
bleu_results_2022_handwritten = bleu.compute(predictions=answers_2022_handwritten, references=gt_answers_2022_handwritten)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2020-2022\n{bleu_results_2020_2022}")
# print(f"Bleu results 2022 handwritten\n{bleu_results_2022_handwritten}")


# add results to dataframe
results.loc[len(results)] = ['t5-base', 'full', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'full', '2022 handwritten', np.array(bert_results_2022_handwritten['precision']).mean(), np.array(bert_results_2022_handwritten['recall']).mean(), np.array(bert_results_2022_handwritten['f1']).mean(), bleu_results_2022_handwritten['bleu']]

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


### Base - finetuned - train set halved

In [14]:
model_name_2020 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2020-smaller"
tokenizer_2020 = AutoTokenizer.from_pretrained(model_name_2020, local_files_only=True)
model_2020 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020, local_files_only=True)

model_name_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2022-smaller"
tokenizer_2022 = AutoTokenizer.from_pretrained(model_name_2022, local_files_only=True)
model_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2022, local_files_only=True)

model_name_2020_2022 = f"{local_models_path}/t5-base-finetuned-squadv2-finetuned-NLB-QA-2042-smaller_combined"
tokenizer_2020_2022 = AutoTokenizer.from_pretrained(model_name_2020_2022, local_files_only=True)
model_2020_2022 = AutoModelForSeq2SeqLM.from_pretrained(model_name_2020_2022, local_files_only=True)

answers_2020 = [get_answer(question, context, tokenizer_2020, model_2020) for question, context in zip(test_data_2020_full["question"], test_data_2020_full["context"])]
answers_2022 = [get_answer(question, context, tokenizer_2022, model_2022) for question, context in zip(test_data_2022_full["question"], test_data_2022_full["context"])]
answers_2020_2022 = [get_answer(question, context, tokenizer_2020_2022, model_2020_2022) for question, context in zip(test_data_2020_2022["question"], test_data_2020_2022["context"])]

# bertscore
bert_results_2020 = bertscore.compute(predictions=answers_2020, references=gt_answers_2020_full, lang="en")
bert_results_2022 = bertscore.compute(predictions=answers_2022, references=gt_answers_2022_full, lang="en")
bert_results_2020_2022 = bertscore.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022, lang="en")
# print(f"Bertscore results 2020\nF1: {np.array(bert_results_2020['f1']).mean()}, Precision: {np.array(bert_results_2020['precision']).mean()}, Recall: {np.array(bert_results_2020['recall']).mean()}")
# print(f"Bertscore results 2022\nF1: {np.array(bert_results_2022['f1']).mean()}, Precision: {np.array(bert_results_2022['precision']).mean()}, Recall: {np.array(bert_results_2022['recall']).mean()}")
# print(f"Bertscore results 2020-2022\nF1: {np.array(bert_results_2020_2022['f1']).mean()}, Precision: {np.array(bert_results_2020_2022['precision']).mean()}, Recall: {np.array(bert_results_2020_2022['recall']).mean()}")

# bleu
bleu_results_2020 = bleu.compute(predictions=answers_2020, references=gt_answers_2020_full)
bleu_results_2022 = bleu.compute(predictions=answers_2022, references=gt_answers_2022_full)
bleu_results_2020_2022 = bleu.compute(predictions=answers_2020_2022, references=gt_answers_2020_2022)
# print(f"Bleu results 2020\n{bleu_results_2020}")
# print(f"Bleu results 2022\n{bleu_results_2022}")
# print(f"Bleu results 2020-2022\n{bleu_results_2020_2022}")


# add results to dataframe
results.loc[len(results)] = ['t5-base', 'smaller', '2020', np.array(bert_results_2020['precision']).mean(), np.array(bert_results_2020['recall']).mean(), np.array(bert_results_2020['f1']).mean(), bleu_results_2020['bleu']]
results.loc[len(results)] = ['t5-base', 'smaller', '2022', np.array(bert_results_2022['precision']).mean(), np.array(bert_results_2022['recall']).mean(), np.array(bert_results_2022['f1']).mean(), bleu_results_2022['bleu']]
results.loc[len(results)] = ['t5-base', 'smaller', '2020-2022', np.array(bert_results_2020_2022['precision']).mean(), np.array(bert_results_2020_2022['recall']).mean(), np.array(bert_results_2020_2022['f1']).mean(), bleu_results_2020_2022['bleu']]

c:\Users\rjutr\miniconda3\envs\project_ds_2\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


## Display all results

In [15]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results)

,Model,Train Data,Data Type,Bert.Precision,Bert.Recall,Bert.F1,BLEU
0,t5-small,None,2020,0.968748,0.972370,0.970398,0.720917
1,t5-small,None,2022,0.951183,0.951029,0.950949,0.648030
2,t5-small,None,2020-2022,0.950706,0.955377,0.952809,0.554557
3,t5-small,None,2022 handwritten,0.894849,0.863893,0.878725,0.067425
4,t5-small,full,2020,0.975338,0.981546,0.978331,0.706494
5,t5-small,full,2022,0.972237,0.976406,0.974130,0.693284
6,t5-small,full,2020-2022,0.970261,0.981644,0.975732,0.543063
7,t5-small,full,2022 handwritten,0.922604,0.889278,0.905240,0.221080
8,t5-small,smaller,2020,0.973396,0.979162,0.976163,0.706436
9,t5-small,smaller,2022,0.964420,0.966231,0.965146,0.695172
